# GinZaの紹介
本紹介は，ツールの歴史的背景や詳細な技術的・構成的情報は簡素化し，ツール利用者向けのチュートリアルを指向しています．ご了承ください．

## GinZaとは
GinZaは，Universal Dependenciesに基づく日本語自然言語処理ライブラリです．MeCab等と同様に形態素解析もできますが，その解析にAI技術（例：Transformers事前学習モデル）を応用することにより高精度化を達成しています．自動で**固有表現抽出**や**係り受け解析**もできます．

ここでUniversal Dependenciesとは，多国語間を横断して一貫性のある注釈を目指した，統語構造の注釈スキームです．平易に言えば，文章構造を分析（形態素解析や品詞分類，係り受け解析など）したときの結果に対し，様々な国が使う言語に依らず同様な注釈（語への品詞付与等）をできるようにしようとする仕組みです．

## GinZa入門

ここでは，本資料執筆時点のPyPIに掲載済みの最新版である``ja_ginza_electra``を紹介します．本バージョンはTransforms事前学習モデルを導入したもので，精度向上を図ったものになります．なお，執筆時点ではβ版ではあるものの``ja_ginza_electra``よりも高精度な解析結果が得られる``ja_ginza_bert_large``が公開されています．

In [ ]:
# ginzaのインストール
!pip install ja_ginza_electra

In [ ]:
# GinZaの使用例：詳細な説明は次のセルから．
import spacy
nlp = spacy.load('ja_ginza_electra')
doc = nlp('信州大学工学部は長野市にあります。')
for sent in doc.sents:
    for token in sent:
        print(
            token.i,
            token.orth_,
            token.lemma_,
            token.norm_,
            token.morph.get("Reading"),
            token.pos_,
            token.morph.get("Inflection"),
            token.tag_,
            token.la
            token.dep_,
            token.head.i,
        )

GinZaは，Universal Dependenciesによる自然言語処理フレームワークであるspaCyを利用して実現されています．そのため，上記のプログラム例では，GinZaの利用はspaCyのライブラリをインポートするところから始まります．なお，プログラミング上はspaCyのAPIを使うことになりますので，APIはspaCyの公式サイト (cf. https://spacy.io/api ) である程度調べることができます．（ある程度と書いているのは，GinZaの実装に依存して得られるデータが決まっている箇所もあるためです．）そして，``spacy.load``関数で引数にGinZaのパッケージ名を指定することでGinZaを読み込みます．このとき得られるデータは，``Language``オブジェクトになります．

ここからは，基本的な使い方を紹介します．``Language``オブジェクト（プログラム上では変数nlp）に対して解析したい文字列を与え（例：``nlp("私は車を運転します。")``て，実行します．この実行結果は``Doc``オブジェクト（ここでは変数doc）として返されます．

``Doc``オブジェクトはトークン（文章の分割単位：実態は``Token``オブジェクト）のシーケンスを持ちます．このトークンを文単位にまとめて取り出すということが，``doc.senets``プロパティを参照することでできます．このようにして取り出したデータが``Span``オブジェクトです．

``Span``オブジェクト（ここでは変数sent）を繰り返し（``for token in sent:``）の中で参照すると，文単位の``Token``オブジェクトのシーケンスに対して処理ができます.
さて，``Token``オブジェクトには様々な属性があります．一部はMeCabと重複するような属性もありますので，MeCabの紹介資料も参考にしてください．

例題プログラム中で参照している属性等について：

``i``は当該トークンの文書中のインデックスを表します．

``orth_``は表記上の語です．

``lemma_``は語の基本形を表します．

``norm_``は基本形で表現を正規化したものです．たとえば，「する」を「為る」といった表現に正規化します．GinZaが形態素解析に用いている``SudachiPy``（正確には``Sudachi``）の説明では，正規化では，送り違い，字種，異体字，誤用，縮約を正規化します．言い換えると，本質的には同じ意味を持つ，異なる複数の表記（誤りを含む）を一つの表記に統一する，というイメージを持つとわかりやすいでしょうか．

``morph.get("Reading")``は表記上の読み仮名を得ます．

``pos_``では，Universal DependenciesのユニバーサルPOSタグに基づく品詞分類を得ます．

``morph.get("Inflection")``では，活用の型を得ます．

``tag_``では，日本語における品詞分類を得ます．

``dep_``では依存関係（文上における役割：たとえば名詞が主語なのか目的語なのかの区別）を得ます．

``head.i``では，単語間の係り受け関係において，当該の語が修飾する先の語のインデックスを示しています．たとえば，プログラム例の結果では「信州大学」は「工学部」を修飾し，「工学部」は「あり（ある）」を修飾しています．

上記のプログラム例の実行結果では，「信州大学」が単語として捉えられています．（形態素とするならば「信州」と「大学」で分かれてもよいでしょう．）その品詞分類を見ると「名詞-**固有名詞**-一般」とあります．このようにGinZaでは固有名詞を捉えることがある程度できるようになっています．

ginzaコマンドを実行することで CoNLL-U Syntactic Annotation 形式 (cf. https://universaldependencies.org/format.html#syntactic-annotation )で出力を得ることもできます．

In [ ]:
#このセルを実行して出力（警告）が得られたら，その最終行が（わかりにくいが）入力フォーム（テキストボックス）になっている．
#その入力フォームに解析したい文章を入力すると解析結果が得られる．停止する場合は，このセル左の停止アイコン（四角）を選択する．
#漢字変換の確定が入力の確定に誤認識される可能性があるため，解析対象の文字列はコピペで入力した方が良い．
!ginza

MeCabの紹介資料で登場した文章例をGinZaにも適用してみます．

In [ ]:
# 長文テキストの解析をデモする文章例
!wget 'https://www.aozora.gr.jp/cards/000148/files/773_14560.html' -O 'bunko.html' #こころ（青空文庫）

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
import spacy

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要
#そのtxtファイルをopenして文字列を取り出すのみで十分
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．

base_list = []
nlp = spacy.load('ja_ginza_electra')

text_splits = text.split() # 今回のバージョンでは，GinZaが一度に処理できるのは49149 bytesまでなので文章を適宜分割
text_list = [' '.join(text_splits[i:i+40]) for i in range(0, len(text_splits), 40)] # 時間がかかるので空白区切りの40文章ずつを繋げたリストを用意

now = 0
for text_unit in text_list: #実行完了まで10分弱かかる可能性がある
    now += 1
    print("現在の進捗：" + str(now) + "/" + str(len(text_splits)//40 + 1))
    doc = nlp(text_unit)
    for sent in doc.sents:
        for token in sent:
            base_list.append(token.norm_)

#語の正規形を収集したリストに対しカウントし辞書化する
count = Counter(base_list)
#カウント数を降順として辞書をソートする
sorted_count = sorted(count.items(), key = lambda word:word[1], reverse=True)
print(dict(sorted_count))


上記プログラムの出力結果に見られるように，頻出語の上位として
- ``{'の': 6195, 'た': 5661, '。': 4654, 'は': 4186, 'て': 3862, 'に': 3707, '、': 3643, 'を': 3221, 'だ': 3054, 'と': 2692,``

が得られました．MeCabの場合，
- ``{'の': 5982, 'た': 5523, '。': 4647, 'は': 4181, 'て': 3856, 'に': 3699, '、': 3610, 'を': 3223, 'だ': 3175, 'と': 2698,``

となり，MeCabと異なるカウントになっていることからわかるように，文章の分割結果がMeCabと異なっています．単純な文例では気づきにくいですが，アルゴリズムや辞書，モデルにより統計結果が大きく異なりうることに注意しましょう．MeCabと同様に品詞による語のフィルタリングやストップワード処理を実行します．

In [ ]:
#ストップワードとして公開されているテキストを取得
!wget 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt' -O 'stopword.txt'

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
import spacy

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要です．
#そのtxtファイルをopenして文字列を取り出すのみで十分でしょう．
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．
#ストップワードの読み込み，改行区切りでの単語分割
stopwords = open("stopword.txt", 'r').read().split('\n')
#カウントする形態素の品詞指定
allowed_pos = ["名詞"]

tag_noun_list = []
pos_noun_list = []
tag_verb_list = []
pos_verb_list = []

nlp = spacy.load('ja_ginza_electra')

text_splits = text.split() # 今回のバージョンでは，GinZaが一度に処理できるのは49149 bytesまでなので文章を適宜分割
text_list = [' '.join(text_splits[i:i+40]) for i in range(0, len(text_splits), 40)] # 時間がかかるので空白区切りの40文章ずつを繋げたリストを用意

now = 0
for text_unit in text_list: #実行完了まで10分弱かかる可能性がある
    now += 1
    print("現在の進捗：" + str(now) + "/" + str(len(text_splits)//40 + 1))
    doc = nlp(text_unit)
    for sent in doc.sents:
        for token in sent:
            if "名詞" in token.tag_ : #日本語の品詞分類に名詞という単語が含まれていたら
                tag_noun_list.append(token.norm_)
            if "NOUN" == token.pos_ : #ユニバーサル品詞タグがNOUN（名詞）であったら
                pos_noun_list.append(token.norm_)
            if "動詞" in token.tag_ : #日本語の品詞分類に動詞という単語が含まれていたら
                tag_verb_list.append(token.norm_)
            if "VERB" == token.pos_ : #ユニバーサル品詞タグがVERB（動詞）であったら
                pos_verb_list.append(token.norm_)

#語の正規形を収集したリストに対しカウントし辞書化する
count_tn = Counter(tag_noun_list)
count_pn = Counter(pos_noun_list)
count_tv = Counter(tag_verb_list)
count_pv = Counter(pos_verb_list)
#カウント数を降順として辞書をソートする
sorted_count = sorted(count_tn.items(), key = lambda word:word[1], reverse=True)
print("Tag_Noun:" + str(dict(sorted_count)))
sorted_count = sorted(count_pn.items(), key = lambda word:word[1], reverse=True)
print("Pos_Noun:" + str(dict(sorted_count)))
sorted_count = sorted(count_tv.items(), key = lambda word:word[1], reverse=True)
print("Tag_Verb:" + str(dict(sorted_count)))
sorted_count = sorted(count_pv.items(), key = lambda word:word[1], reverse=True)
print("Pos_Verb:" + str(dict(sorted_count)))


名詞による語のフィルタリング，ならびにストップワード処理により，実行結果は以下のようになりました．

- 日本語の品詞分類における名詞：``{'私': 2676, '其れ': 638, '先生': 597, '事': 576, 'k': 411, '物': 401, '時': 390, '奥さん': 386,``
- ユニバーサルPOSタグにおける名詞：``{'先生': 597, '事': 576, '物': 411, 'k': 411, '時': 390, '奥さん': 386, '父': 269, '自分': 261,``
- 日本語の品詞分類における動詞：``{'た': 5661, 'だ': 3054, '為る': 2042, 'ます': 1675, 'です': 1625, '居る': 1367, 'ない': 1170,``
- ユニバーサルPOSタグにおける動詞：``{'居る': 1363, '言う': 1005, '為る': 834, '有る': 811, '成る': 545, '来る': 342, '見る': 294,``


（参考）MeCabで収集した名詞：``{'先生': 597, None: 437, '奥': 401, '父': 296, '母': 184, '嬢': 168, '顔': 133, '言葉': 124, '二人': 115, '眼': 111, '心': 106``

日本語の品詞分類における名詞では「私」や「其れ」が頻出語として抽出されていますが，ユニバーサルPOSタグにおける名詞では抽出されていません．これは後者が代名詞（PRON）や固有名詞（PROPN）を名詞（NOUN）とは区別して扱っているからです．（注：細かく条件付けすれば，日本語の品詞分類で代名詞や固有名詞は区別できます．今回のような単純な条件付けではこのようになるということです．）また，MeCabでは「奥」と言う語が形態素として得られていましたが，GinZaでは「奥さん」と言う（物語上でより自然な）単語で抽出されています．GinZaでは「k」も名詞で捉えられています．

一方，動詞を見ると，日本語の品詞分類では「だ」や「ます」，「です」，「ない」といった確かに動詞ではあるが，意味としては捉えにくいものが挙げられています．ユニバーサルPOSタグでは，「言う」や「有る」，「成る」，「来る」など単独で意味が捉えやすい単語に絞って得られているのも特徴的です．

このように，どのような体系のタグで何の品詞を得るかにより，得られる文書の特徴が変わります．旧来では，文書間の類似度計算などにおいては，文書を特徴づけられる語を如何に適切に抽出できるかが一つの重要な課題であり，汎用性を求めることに限界がありました．現代では，AI技術によりシンプルなプログラムで汎用的な特徴抽出が可能になってきています．

## 単語間の係り受け関係（Syntactic Dependency Relation）の可視化
spaCyでは，文を分割した単語間の依存関係を可視化するモジュールとして``displacy``があります．依存関係の可視化は，大量のテキストを処理する場合には，あまり気にする必要はないように思いますが，解析結果の理解が不十分であれば，理解を促進してくれる効果があるように思います．必要に応じて，利用してみましょう．

``spacy.displacy.render``メソッドで可視化できます．利用できる引数は少し説明しますが，完全なものは https://spacy.io/usage/visualizers を参考にしてください．

styleには'dep'か'ent'か'span'を与えます．'dep'が係り受け関係を描画するモードで，'ent'が固有表現抽出結果を描画するモードになります．'span'は複数語に跨ってラベルづけをするような可視化を提供しますが，ラベルやその範囲は人が指定するようです．また，google colab上で利用するには``jupyter=True``の引数を加えてください．


In [ ]:
import spacy
nlp = spacy.load('ja_ginza_electra')
doc = nlp('信州大学工学部は長野市にあります。')
spacy.displacy.render(doc, style='dep', jupyter=True)

可視化結果の矢印は，被修飾語から修飾語へと伸びています．

## 固有表現（Named Entity）の抽出と可視化
``displacy``では，上記と同様な方法で固有表現を可視化することもできます．必要に応じて，利用してみましょう．

In [ ]:
import spacy
nlp = spacy.load('ja_ginza_electra')
doc = nlp('ジョンはジブリ映画と秋葉原が大好きです。今日は午後８時にブランドシネマという映画館に行き，マリーと一緒に映画を見る予定です。')

spacy.displacy.render(doc, style='ent', jupyter=True)

プログラム上で利用できるように固有表現を抽出する方法としては，docのentsを参照する方法があります．entsの各要素(ここでは変数``ent``)に対し，``label_``でラベルを得られます．

In [ ]:
# 固有表現を抽出するプログラム例
import spacy
nlp = spacy.load('ja_ginza_electra')
doc = nlp('ジョンはジブリ映画と秋葉原が大好きです。今日は午後８時にブランドシネマという映画館に行き，マリーと一緒に映画を見る予定です。')
for ent in doc.ents:
    print(
        ent.text,
        ent.label_,
    )

## おまけ：LLM (Large Language Model)による固有表現（Named Entity）の抽出
執筆時点では，spaCyはLLMと連携して動作させることが可能です．ここでは　おまけ　として，その紹介をします．少し導入の敷居が高いかもしれません．まず``spacy-llm``をインストールします．ランタイムは，``ランタイム``メニューから``ランタイムのタイプの変更``を通じてGPU (T4 GPU)にしておいてください．

In [ ]:
!pip install spacy-llm

次にLLMとの連携設定を書いた``config.cfg``と名付けたファイルを作り，下記の内容とします．ここで，本例ではGoogleのPaLM APIを利用することを前提とします．PaLM APIのAPI Keyが必要になりますので， https://developers.generativeai.google/tutorials/setup?hl=ja を経由してAPI Keyを取得してください．``API キーを取得する``を選択し，移動先で``Create API key in new project``を選択する流れでKeyが取得できます．

Config.cfg
```
[nlp]
lang = "en"
pipeline = ["llm"]

[components]

[components.llm]
factory = "llm"

[components.llm.task]
@llm_tasks = "spacy.NER.v3"
labels = ["PERSON", "ORGANISATION", "LOCATION"]

[components.llm.model]
@llm_models = "spacy.PaLM.v1"
config = {"temperature": 0.0}
```

次に環境変数にPALM_API_KEYという名でAPI Keyを設定する必要があるため，以下の``<API_KEY>``に具体的なキーを与えて実行します．

注：API Keyの意図しない漏洩リスクを少しでも減らすため，Google Colabではなく，ローカル環境で実行環境を整えて実行されることを推奨します．一方で，Google Colab上で実行可能なことは実証済みです．

In [ ]:
%env PALM_API_KEY <API_KEY>

本例では，固有表現（NER）抽出を試みます．config.cfgにおいて``[components.llm.task]``内の``labels = ["PERSON", "ORGANISATION", "LOCATION"]``をラベルとして，固有表現を抽出します．（どうやら現状，ラベルを4種以上にすると上手く抽出できないようです．）

In [ ]:
import spacy
from spacy_llm.util import assemble

nlp = assemble("config.cfg") # config.cfgを読み込んでLLMと連携
doc = nlp('John loves Ghibli movies and Akihabara. Today we will go to a movie theater called Brando Cinema at 8 p.m. and watch a movie with Marie.')
spacy.displacy.render(doc, style='ent', jupyter=True)

結果としては，GhibliとBrando CinemaがORGANISATIONとされ，AkihabaraがLocationとされ，MarieがPERSONとされました．しかし，Johnが上手くPERSONとして認識されません．より精度よく結果を得るには，調整が必要そうです．

## 演習
1. 手持ちの文書に対し，GinZaを利用し，日本語の品詞分類とユニバーサルPOSタグでどちらが文書の特徴を捉えられているが，頻出語を計算するなどして確認しなさい．
1.  手持ちの文書に対し，固有表現抽出を行ない，固有表現が適切に抽出されているか評価しなさい．

### 参考文献
1. Megagon Labs, GiNZA: 日本語自然言語処理オープンソースライブラリ, https://www.megagon.ai/jp/projects/ginza-install-a-japanese-nlp-library-in-one-step/
1. megagonlabs, GiNZA - Japanese NLP Library, https://megagonlabs.github.io/ginza/
1. Universal Dependencies contributors, Universal Dependencies, https://universaldependencies.org/
1. Universal Dependencies contributors, UD for Japanese, https://universaldependencies.org/ja/index.html
1. Explosion, spaCy, https://spacy.io/
1. Explosion, spaCy Library Architecture, https://spacy.io/api
1. PyPi, SudachiPy 0.6.7, https://pypi.org/project/SudachiPy/
1. WorksApplications, SudahiPy, https://github.com/WorksApplications/sudachi.rs/tree/develop/python
1. WorksApplications, Sudahi, https://github.com/WorksApplications/Sudachi
1. PyPi, spacy-llm 0.6.4, https://pypi.org/project/spacy-llm/
1. 青空文庫, 青空文庫, https://www.aozora.gr.jp/
